In [1]:
import json
import os

# Paths
coco_json_path = "/Users/laman/Desktop/coco/task_ayan done_4_annotations_2025_01_02_11_35_27_coco 1.0.json"
output_dir = "/Users/laman/Desktop/coco/labels"
images_dir = "/Users/laman/Desktop/coco/imgs"

# Load COCO JSON
with open(coco_json_path, 'r') as f:
    coco_data = json.load(f)

# Create directories
os.makedirs(output_dir, exist_ok=True)
labels_dir = os.path.join(output_dir, "labels")
os.makedirs(labels_dir, exist_ok=True)

# Build dict for quick lookup: image_id -> (filename, width, height)
image_info = {}
for img in coco_data["images"]:
    image_info[img["id"]] = {
        "file_name": img["file_name"],
        "width": img["width"],      # get the actual width
        "height": img["height"]     # get the actual height
    }

# Build dict for category id -> category name
categories = {}
for cat in coco_data["categories"]:
    categories[cat["id"]] = cat["name"]

# Process annotations
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    category_id = ann["category_id"]
    bbox = ann["bbox"]  # in COCO format: [x_min, y_min, width, height]

    # Retrieve actual image width & height
    img_w = image_info[image_id]["width"]
    img_h = image_info[image_id]["height"]

    # Convert COCO -> YOLO
    x_min, y_min, w, h = bbox
    x_center = x_min + w / 2.0
    y_center = y_min + h / 2.0

    # Normalize
    x_center /= img_w
    y_center /= img_h
    w /= img_w
    h /= img_h

    # YOLO expects: class x_center y_center width height
    # Usually YOLO class indices are zero-based; COCO cat ids might not be
    yolo_class_id = category_id - 1  # or another mapping if needed

    # Write to .txt file
    image_name = image_info[image_id]["file_name"]
    txt_file_name = os.path.splitext(image_name)[0] + ".txt"
    txt_path = os.path.join(labels_dir, txt_file_name)
    with open(txt_path, 'a') as f_txt:
        f_txt.write(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# Create a data.yaml
classes = [categories[k] for k in sorted(categories.keys())]
with open(os.path.join(output_dir, "data.yaml"), 'w') as yaml_file:
    yaml_file.write(f"train: {images_dir}\n")
    yaml_file.write(f"val: {images_dir}\n")
    yaml_file.write(f"nc: {len(classes)}\n")
    yaml_file.write(f"names: {classes}\n")

print(f"YOLO annotations saved in {output_dir}")


YOLO annotations saved in /Users/laman/Desktop/coco/labels
